In [1]:
import pandas as pd
import pickle

In [2]:
path_model = open('models/titanic1.pkl', 'rb')
model = pickle.load(path_model)

In [3]:
test_data = pd.read_csv('test.csv')

In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
selected_features = test_data.loc[:, ['Pclass', 'Sex', 'SibSp', 'Parch', 'Age', 'Embarked']]

In [6]:
selected_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass      418 non-null int64
Sex         418 non-null object
SibSp       418 non-null int64
Parch       418 non-null int64
Age         332 non-null float64
Embarked    418 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 19.7+ KB


In [7]:
selected_features.Age.fillna(selected_features.Age.median(),  inplace=True)

In [8]:
## grouping age
def age_grouping(data):
    if(data<4):
        return 'Inf_toddler'
    elif(data<=12):
        return 'children'
    elif(data<20):
        return 'teens'
    elif(data<45):
        return 'young_middle'
    else:
        return 'elders'

In [9]:
selected_features['Age_group'] = selected_features.Age.apply(age_grouping)
selected_features.drop('Age', axis=1, inplace=True)

In [10]:
selected_features.head()

,Pclass,Sex,SibSp,Parch,Embarked,Age_group
0,3,male,0,0,Q,young_middle
1,3,female,1,0,S,elders
2,2,male,0,0,Q,elders
3,3,male,0,0,S,young_middle
4,3,female,1,1,S,young_middle


In [11]:
selected_features = pd.get_dummies(selected_features, drop_first=True)

In [12]:
selected_features.head()

,Pclass,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S,Age_group_children,Age_group_elders,Age_group_teens,Age_group_young_middle
0,3,0,0,1,1,0,0,0,0,1
1,3,1,0,0,0,1,0,1,0,0
2,2,0,0,1,1,0,0,1,0,0
3,3,0,0,1,0,1,0,0,0,1
4,3,1,1,0,0,1,0,0,0,1


In [13]:
predictions = model.predict(selected_features)

In [14]:
len(predictions)

418

In [15]:
pred_data = pd.concat([test_data.PassengerId, pd.Series(predictions)], axis=1)

In [16]:
pred_data.columns = ['PassengerId', 'Survived']

In [17]:
pred_data.to_csv(path_or_buf='Kaggle_upload/titanic_survive_prediction', sep=',', index=False)